# Start Interactive Dask Cluster

In [ ]:
pip install --upgrade dask-cloudprovider

In [ ]:
pip install --upgrade azureml-sdk

**RESTART YOUR KERNEL**

## Workspace

Get the Azure ML workspace.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

### Start cluster

In [ ]:
from dask_cloudprovider import AzureMLCluster

vm_size = 'STANDARD_DS13_V2'
timeout = 7200 

cluster = AzureMLCluster(ws, vm_size=vm_size, scheduler_idle_timeout=timeout, jupyter=True, show_output=True)

In [ ]:
cluster.run

In [ ]:
cluster.scale(20) # need more than default quota for this 

In [ ]:
# print out cluster widget 
cluster

In [ ]:
#cluster.close()

In [ ]:
help(AzureMLCluster)